# HW2: ASR

Скачиваем все необходимые библиотеки

In [1]:
%%capture
!pip install --upgrade --force-reinstall 'git+https://github.com/ytdl-org/youtube-dl.git'
!pip install SpeechRecognition
!pip install 'git+https://github.com/openai/whisper.git' soundfile
!pip install assemblyai
!pip install pocketsphinx
!pip install jiwer

Импорты

In [2]:
import re
import jiwer
import torch
import torchaudio
import speech_recognition as sr
import assemblyai as aai
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

Эталон для субтитров:

In [3]:
golden_subtitles = \
'''
Ike was born on Christmas Day in 1642,
In England’s bonnie Lincolnshire is where that child grew.
At 18 he went off to be at Cambridge University;
His passion there for Mathematics grew –
In years before the Calculus Priority Dispute.

Now, Will was born in Germany when Ike was only three;
By 16 he had earned a university degree.
In Law he earned an LL.D., then worked awhile in Alchemy;
But math was not a path that he pursued –
In years before the Calculus Priority Dispute.

Cambridge halls were closed by plague in 1665;
So Ike returned to Lincolnshire with fresh and fertile mind.
For two years he worked avidly on Calculus and gravity
But of his thoughts he published just a few –
A fault that spawned the Calculus Priority Dispute.

Issac shared his opus with his Cambridge teacher-friend
Who sent it to another to show what his pupil penned.
When Willie met this second guy, Ike’s manuscript did he espy;
The notes he took would tarnish his repute –
And further fuel the Calculus Priority Dispute.

The learned British army pledged allegiance to their man
While Willie earned support from other nations in the land.
And mud was slung by either side, defending Ike or Willie’s pride
As Europe’s academic battle brewed –
The continental Calculus Priority Dispute.

Responding to suggestions that Will’s honour was corrupt
He asked the Royal Society to clear the matter up.
That jury was presided by none other than that Newton guy;
The def’rence of the group was absolute –
Which added to the Calculus Priority Dispute.

Now, Willie spent his final years just trying to clear his name;
Emotionally damaged from this vengeful, bitter game.
I don’t know if he laughed or cried when Isaac heard his “buddy” died,
But certainly it gave him an acute –
… advantage in the Calculus Priority Dispute.

The scoreboard on this battlefield with centuries traversed
Says Will worked independently but Isaac got there first;
Though Willie clearly takes the prize for symbolism he devised.
When all the points are tallied, most conclude:
Two winners in the Calculus Priority Dispute.

When all the points are tallied, most conclude:
Two winners in the Calculus Priority Dispute.
'''

В эталоне меняем символ переноса строки на пробел:

In [4]:
golden_subtitles = golden_subtitles.replace('\n', ' ')

Инициализируем трансформации для jiwer (нижний регистр, удаление лишних пробелов и пунктуации, токенизация):

In [5]:
transformation = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.RemovePunctuation(),
    jiwer.ReduceToListOfListOfWords(word_delimiter=' ')
])

## 1. Youtube Autogenerated

Скачиваем видео с субтитрами с помощью youtube-dl:

In [6]:
!youtube-dl --write-auto-sub -x --audio-format wav --output '%(title)s.%(ext)s' 'https://www.youtube.com/watch?v=ryai59Gta-4'

[youtube] ryai59Gta-4: Downloading webpage
[info] Writing video subtitles to: The Calculus Priority Dispute.en.vtt
[dashsegments] Total fragments: 1
[download] Destination: The Calculus Priority Dispute.m4a
[download] 100% of 3.69MiB in 00:00
[ffmpeg] Correcting container in "The Calculus Priority Dispute.m4a"
[ffmpeg] Destination: The Calculus Priority Dispute.wav
Deleting original file The Calculus Priority Dispute.m4a (pass -k to keep)


Превращаем vtt формат в одну строку:

In [7]:
bad_words = ['-->', '</c>', 'WEBVTT', 'Kind: captions', 'Language: en']

youtube_auto_subtitles = ''

with open('The Calculus Priority Dispute.en.vtt') as oldfile:
    for line in oldfile:
        if not any(bad_word in line for bad_word in bad_words):
            youtube_auto_subtitles += line.strip('\n ')

youtube_auto_subtitles

"Ike was born on Christmas Day in 1642  England's bonny Lincolnshire is where  that child through 18 he'll niversity  his passion there for mathematics in  years before the calculus priority  disputes now loses folding Germany when  I was only 3 by 16 the other the  university degree law he earns and Pell  L leave and worked a while in alchemy  maths is not a path that he pursues  in years before the calculus priority  dispute  Gate Bridge hulls were closed my plague  in 1665 so Mike returned to Lincolnshire  and fresh and fertile mind for two years  he worked avidly on calculus and gravity  and all his thoughts he published just a  few that spawned the calculus priority  dispute Isaac's shared his office with  his shame bridge dear friend who sent it  to another to show what is nubile pen  when Willie met this second guy I expect  you script if he is five the notes we  took with our dish is repute and further  view the calculus priority dispute  learning the British Hartness pledged  

In [8]:
wer_youtube = jiwer.wer(
    golden_subtitles,
    youtube_auto_subtitles,
    truth_transform=transformation,
    hypothesis_transform=transformation
)
print(f'Youtube WER: {wer_youtube}')

Youtube WER: 0.30578512396694213


## 2. Speech Recognition (Whisper)

In [9]:
r = sr.Recognizer()
with sr.AudioFile('The Calculus Priority Dispute.wav') as source:
    audio = r.record(source)
whisper_subtitles = r.recognize_whisper(audio)

In [10]:
whisper_subtitles

" I was born on Christmas Day in 1642. England's funny Lincolnshire is where that child grew. 18, he went up to see a Cambridge University. His passion never met the man it grew. In years before the calculus priority dispute. Now, when was born in Germany when I was only three? By sixteen, he had earned the University degree. In Laughy earned and LLD then worked a while in Althamies, but math was not a path that he pursued. In years before the calculus priority dispute. Cambridge halls were closed by plague in 1665. So I returned to Lincolnshire with fresh and fertile mind. For two years he worked avidly on calculus and gravity, but on his thoughts he published just a few. A bump that spawned the calculus priority dispute. Isaac shared his hopeless British teacher friend, sent it to another to show what his pupil had. When Willie met this second guy, I expected his rapid bias by the notes he took with punished his repute. And further fueled the calculus priority dispute. He learned Bri

In [11]:
wer_whisper = jiwer.wer(
    golden_subtitles,
    whisper_subtitles,
    truth_transform=transformation,
    hypothesis_transform=transformation
)
print(f'Whisper WER: {wer_whisper}')

Whisper WER: 0.15977961432506887


## 3. Speech Recognition (Sphinx)

In [12]:
sphinx_subtitles = r.recognize_sphinx(audio)
sphinx_subtitles

"there there there there i was old and you live one evening in finland and i'm at a senior and a year when the white house and manning's it on new year's in london and nile and you can now move all of it and my faith and the and you cannot use them and live while the one man on and in the years only known as thine own needs and now they're none loopholes one way to he of the nine hundred and earn behind year's eve where there are enough to when the listens in and all the long enough and you there in nineteen years the things that bring them to send you one hundred one in the leg believe and an knife and you find you and kind of nine and need new year there there if and a half the one in bought with what i thrive on thigh and you live in them and and and the the cotton in and lyle and there and there's a long time and what what if what time for one side by side of time of death and lives with me and i live on io and you can in lily's it is mine and that is on me and evolution of the fam

In [13]:
wer_sphinx = jiwer.wer(
    golden_subtitles,
    sphinx_subtitles,
    truth_transform=transformation,
    hypothesis_transform=transformation
)
print(f'Sphinx WER: {wer_sphinx}')

Sphinx WER: 0.9449035812672176


## 4. assemblyai

In [14]:
aai.settings.api_key = '5469a26468cb4116a27553ee7312881f'

transcriber = aai.Transcriber()
transcript = transcriber.transcribe('The Calculus Priority Dispute.wav')

assemblyai_subtitles = transcript.text
assemblyai_subtitles

"You. Ike was born on Christmas Day in 1642. England's Bonnie Lincolnshire is where that child grew. At 18, he went off to be at Cambridge University. His passion there for mathematics grew in years before the calculus priority dispute. Now, Will was born in Germany when Ike was only three. By 16 he had earned a university degree in law. He earned an LLD, then worked a while in alchemy. But math was not a path that he pursued. In years before the calculus priority dispute, Cambridge halls were closed by plague in 1665, so Ike returned to Lincolnshire with fresh and fertile mind. For two years he worked avidly on calculus and gravity. But of his thoughts he published just a few a fault that bond the calculus priority dispute Isaac shared his opus with his Cambridge teacher friend, who sent it to another to show what his pupil penned. When Willie met this second guy. Ike's manuscripted pspy, the notes he took would tarnish his repute and further fueled the calculus priority dispute. The 

In [15]:
wer_assemblyai = jiwer.wer(
    golden_subtitles,
    assemblyai_subtitles,
    truth_transform=transformation,
    hypothesis_transform=transformation
)
print(f'assemblyai WER: {wer_assemblyai}')

assemblyai WER: 0.06887052341597796


Результаты:

4: у Sphinx получилась абракадабра (WER ~0.94);

3: автоматически сгенерированные YouTube субтитры посредственные по качеству (WER ~31);

2: результат Whisper лучше, чем автогенерация YouTube (WER ~0.16);

1: результат AssemblyAI почти идеален (очень маленький WER ~0.07).